# Discovering Sentinel-2 products

In [ ]:
%load_ext autoreload
%autoreload 2

from eo_tools.util import visualize_S2_products

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import json

import geopandas as gpd
import folium
import folium.plugins
import logging
import numpy as np

import shapely
logging.basicConfig(level=logging.INFO)

## Select an area of interest

- Select a drawing tool (polygon, circle or rectangle)
- Draw the AOI on the map
- Click "Export" on the right to save the GeoJSON file

In [ ]:
map_select = folium.Map()
folium.plugins.Draw(
    export=True,
    draw_options={
        "polyline": False,
        "circlemarker": False,
        "marker": False,
    },
).add_to(map_select)
map_select

## Search

In [ ]:
# load a geometry
file_aoi = "/data/aoi_Maroc.geojson"
# file_aoi = "/data/aoi_bretagne.geojson"
gj_aoi = read_geojson(file_aoi)
aoi = geojson_to_wkt(gj_aoi["features"][0])
shp = gpd.read_file(file_aoi).geometry[0]

# open credentials
with open("/data/creds.json") as f:
    creds = json.load(f)

# search for products intersecting geometry
api = SentinelAPI(creds["username"], creds["password"])
products = api.query(
    aoi,
    platformname="Sentinel-2",
    date=("20230901", "20230920"),
    #  orbitdirection='ASCENDING',
    limit=50,
)

# make a geodataframe for visualization
gdf = api.to_geodataframe(products).reset_index(names='id')

In [ ]:
# gdf.geometry[1:3].explore()
# gdf[gdf.s2datatakeid=='GS2A_20230908T110621_042891_N05.09'].geometry.explore()
shapely.geometry.mapping(gdf[gdf.s2datatakeid=='GS2A_20230908T110621_042891_N05.09'].unary_union)
# np.unique(gdf.s2datatakeid)
# for g in gdf.groupby('s2datatakeid'):
    # print(len(g[1]))

## Visualize by groups

- For InSAR, products must be overlapping and from the same relative orbit.
- They also have to be of the same ascending / descending configuration.
- Choosing them from the same slice is the easiest though overlapping slices could be combined too.

In [65]:
visualize_S2_products(gdf, gj_aoi)

## Download

In [ ]:
# fill indices of products to download
indices_to_download = [1, 5]
ids = gdf['id'][indices_to_download]
# dl = api.download_all_quicklooks(ids, directory_path='/data/S1/')
dl = api.download_all(ids, directory_path='/data/S1/')

# Process

In [ ]:
# extract paths for processing
file_paths = [item[1]['path'] for item in dl[0].items()]
process_InSAR(
    file_paths[0],
    file_paths[1],
    out_dir="/data/res",
    tmp_dir="/data/tmp",
    shp=shp,
    pol="VV",
    coh_only=False,
    intensity=True,
    clear_tmp_files=True,
    resume=False
)

In [ ]:
file_phase = "/data/res/phi_VV_03Sep2023_15Sep2023_slice2_crop.tif"
visualize_insar_phase(file_phase)

In [ ]:
file_coh = "/data/res/coh_VV_03Sep2023_15Sep2023_slice2_crop.tif"
visualize_insar_coh(file_coh)

In [ ]:
file_int = "/data/res/int_VV_03Sep2023_slice2_crop.tif"
visualize_sar_intensity(file_int)